In [ ]:
#Script que gera NF, ITEMNF e VENDAS em formaato .xlsx direto na pasta para a importacao

import random
import pandas as pd
import os
from datetime import datetime, date
from sqlalchemy import create_engine, VARCHAR,NUMERIC,INTEGER,DATE,DATETIME,String,text,CHAR
import numpy as np
from faker import Faker

# Quantidade de notas fiscais

numero_inicial_nota = 1000
numero_maximo_nota = 3001
quant_maxima_itens = 10
variavel_controle_maximo_LOOP = 1100 #Valor maximo que da nota inicial dentro do loop de geracao de NF's

fake = Faker()

server = "DESKTOP-HV85ADA\\PROJETO_MERCADO"
bancodados = "MERCADO_OLTP"
driver = "ODBC Driver 17 for SQL Server"

conexaostring = f"mssql+pyodbc://{server}/{bancodados}?driver={driver}&Trusted_Connection=yes"
engine = create_engine(conexaostring, pool_size=10, max_overflow=30)

#Função para testar a conexão
def testar_conexao_sqlalchemy(engine):
    try:        
        with engine.connect() as conexao: #Tenta conectar ao banco de dados
            print("Conexão com o banco de dados estabelecida com sucesso")
            result = conexao.execute(text("SELECT 1 AS teste")) #Executa uma consulta para testar a conexão
            print("Query executada com sucesso")
            return True
    except Exception as e:
        print(f"Erro ao conectar ao banco de dados: {e}")
        return False
#testa a conexão
if testar_conexao_sqlalchemy(engine): 
    print("A conexão foi bem-sucedida")
else:
    print("Falha na conexão.")

# Função para gerar a nota fiscal
def gerar_nota_fiscal(numero_nota, lista_produtos, quant_maxima_itens, data_nota):
    # Garantir que a quantidade de itens únicos seja pelo menos 1
    quant_item_unicos = random.randint(1, quant_maxima_itens)  # Gerar quantidade aleatória de itens únicos na nota fiscal
    #print(f"Quantidade de itens únicos na nota {numero_nota}: {quant_item_unicos}")

    lista_saida = []  # Lista para armazenar os itens da nota fiscal
    lista_NOTAFISCAL = []   

    id_forma_pgmt = random.randint(1, 8)
    total_clientes_db = (engine.connect().execute(text("SELECT COUNT(IDCLIENTE) FROM [MERCADO_OLTP].[dbo].CLIENTE"))).scalar()
    id_cliente = random.randint(1, total_clientes_db)
    total_lojas_db = (engine.connect().execute(text("SELECT COUNT(IDLOJA) FROM [MERCADO_OLTP].[dbo].LOJA"))).scalar()
    id_loja = random.randint(1, total_lojas_db)
    id_cfop = random.randint(1, 6)
    chaveacesso = ''.join([str(random.randint(0, 9)) for _ in range(44)])
    data_criacao = data_nota.date()
    data_atlz = ""
    versao = 1
    
    # Gerar os itens da nota fiscal
    for _ in range(quant_item_unicos):
        produto = random.choice(lista_produtos)
        if produto in df_prod["NOME"].values:
            preco_unit = df_prod.loc[df_prod["NOME"] == produto, "PRECO_UNIT"].values[0]
            id_produto = df_prod.loc[df_prod["NOME"] == produto, "IDPRODUTO"].values[0]
            preco_unit = round(float(preco_unit), 2)  # Preço unitário com 2 casas decimais
            quantidade = random.randint(1, 10)  # Quantidade aleatória do produto
            preco_total_item = round(quantidade * preco_unit, 2)  # Total do item
            lista_saida.append((data_nota, numero_nota, produto, quantidade, preco_unit, preco_total_item, 
                                id_forma_pgmt, id_cliente, id_loja, id_cfop, chaveacesso, data_criacao, 
                                data_atlz, versao,id_produto))            

    # Calcular o total da nota fiscal
    preco_total_nota = round(sum(item[5] for item in lista_saida), 2)
    
    # Adicionar o total da nota a cada item da lista
    lista_saida = [item + (preco_total_nota,) for item in lista_saida]
   
    lista_NOTAFISCAL.append((data_nota, id_forma_pgmt, id_cliente, id_loja, id_cfop, chaveacesso, data_criacao, 
                             data_atlz, versao,preco_total_nota))
    

    # Criar um DataFrame com os dados da nota fiscal
    df_nota_fiscal = pd.DataFrame(
        lista_saida, columns=["DATA_NOTAFISCAL", "Numero_Nota", "Produto", "QUANTIDADE", "PRECO_UNIT", "TOTAL_ITEM", 
                              "ID_FORMA_PGMT", "ID_CLIENTE", "ID_LOJA", "ID_CFOP","CHAVE_ACESSO","DATA_CRIACAO",
                              "DATA_ATLZ","VERSAO", "IDPRODUTO","TOTAL_NOTA"]
    )

    df_2 = pd.DataFrame(
        lista_NOTAFISCAL, columns=["DATA_NOTAFISCAL", "ID_FORMA_PGMT", "ID_CLIENTE", "ID_LOJA", "ID_CFOP", 
                                   "CHAVE_ACESSO", "DATA_CRIACAO", "DATA_ATLZ", "VERSAO","TOTAL_NOTA"]
    )

    
    return df_nota_fiscal, df_2


while numero_inicial_nota < variavel_controle_maximo_LOOP:
    try:
        
        incremento = random.randint(33, 87)
        numero_maximo_nota = numero_inicial_nota + incremento
        df_prod = pd.read_sql(text(f"SELECT IDPRODUTO, NOME, PRECO_UNIT FROM PRODUTO "),engine)
        lista_prod_DB = df_prod['NOME'].to_list()
        df_notas_fiscais = []
        df_notas_fiscais_2 = []
        for numero_nota in range(numero_inicial_nota, numero_maximo_nota + 1):
            data_nota_ini = fake.date_time_between(start_date=datetime(2024,1,1),end_date=datetime(2024,1,20))
            hora_aleatoria = random.randint(7, 21)  #Gera uma hora entre 8 e 22
            minuto_aleatorio = random.randint(0, 59)  #Gera minutos aleatórios
            segundo_aleatorio = random.randint(0, 59)  #Gera segundos aleatórios
            data_nota = data_nota_ini.replace(hour=hora_aleatoria, minute=minuto_aleatorio, second=segundo_aleatorio) #Substituir o horário da data_nota pelo horário gerado
            df_nota_fiscal, df_2 = gerar_nota_fiscal(numero_nota, lista_prod_DB, quant_maxima_itens, data_nota)
            df_notas_fiscais.append(df_nota_fiscal)
            df_notas_fiscais_2.append(df_2)
        df_todas_notas = pd.concat(df_notas_fiscais, ignore_index=True)
        df_todas_notas2 = pd.concat(df_notas_fiscais_2, ignore_index=True)
        df_ITEMNOTA = pd.DataFrame({
                        'ID_NOTAFISCAL':  df_todas_notas['Numero_Nota'],
                        'ID_PRODUTO':     df_todas_notas['IDPRODUTO'],
                        'QUANTIDADE':     df_todas_notas['QUANTIDADE'],
                        'PRECO_UNIT':     df_todas_notas['PRECO_UNIT'],
                        'TOTAL_ITEM':     df_todas_notas['TOTAL_ITEM'],
                        'DATA_CRIACAO':   df_todas_notas['DATA_CRIACAO'],
                        'DATA_ATLZ':      df_todas_notas['DATA_ATLZ'],
                        'VERSAO':         df_todas_notas['VERSAO'],
        })
        df_NOTAFISCAL = pd.DataFrame({
                        'DATA':           df_todas_notas2['DATA_NOTAFISCAL'],
                        'TOTAL_NOTA':     df_todas_notas2['TOTAL_NOTA'],
                        'ID_FORMA_PGMT':  df_todas_notas2['ID_FORMA_PGMT'],
                        'ID_CLIENTE':     df_todas_notas2['ID_CLIENTE'],
                        'ID_LOJA':        df_todas_notas2['ID_LOJA'],
                        'ID_CFOP':        df_todas_notas2['ID_CFOP'],
                        'CHAVE_ACESSO':   df_todas_notas2['CHAVE_ACESSO'],
                        'DATA_CRIACAO':   df_todas_notas2['DATA_CRIACAO'],
                        'DATA_ATLZ':      df_todas_notas2['DATA_ATLZ'],
                        'VERSAO':         df_todas_notas2['VERSAO'],
        })
        df_VENDA = pd.DataFrame({
                        'DATA':           df_todas_notas['DATA_NOTAFISCAL'],
                        'ID_NOTAFISCAL':  df_todas_notas['Numero_Nota'],
                        'ID_CLIENTE':     df_todas_notas['ID_CLIENTE'],
                        'ID_LOJA':        df_todas_notas['ID_LOJA'],
                        'ID_FORMA_PGMT':  df_todas_notas['ID_FORMA_PGMT'],
                        'TOTAL_VENDA':     df_todas_notas['TOTAL_NOTA'],
                        'DATA_CRIACAO':   df_todas_notas['DATA_CRIACAO'],
                        'DATA_ATLZ':      df_todas_notas['DATA_ATLZ'],
                        'VERSAO':         df_todas_notas['VERSAO'],
        })
        caminho = r"C:\\Users\\marce\\OneDrive\Arquivos Antigos\\Área de Trabalho\\Projeto Supermecado - Arquivos\\VENDAS"
        path_ITEMNOTA = r"\\ITEMNOTA\\Novo(a) Documento de Texto.csv"

        # Caminhos das pastas
        pastas = {
            'ITEMNOTA': r"C:\\Users\\marce\\OneDrive\Arquivos Antigos\\Área de Trabalho\\Projeto Supermecado - Arquivos\\VENDAS\\ITEMNOTA",
            'NOTAFISCAL': r"C:\\Users\\marce\\OneDrive\Arquivos Antigos\\Área de Trabalho\\Projeto Supermecado - Arquivos\\VENDAS\\NOTAFISCAL",
            'VENDA': r"C:\\Users\\marce\\OneDrive\Arquivos Antigos\\Área de Trabalho\\Projeto Supermecado - Arquivos\\VENDAS\\VENDA"
        }
        # Dicionário com os DataFrames e suas pastas correspondentes
        dataframes = {
            'ITEMNOTA': df_ITEMNOTA,
            'NOTAFISCAL': df_NOTAFISCAL,
            'VENDA': df_VENDA
        }
        # Salvar cada DataFrame em um arquivo CSV na pasta correspondente
        for nome_df, df in dataframes.items():
            try:
                id_valor = f"vendas_{numero_inicial_nota}ate_{numero_maximo_nota}"
                # Nome do arquivo
                nome_arquivo = f"arquivo_{nome_df}_{id_valor}.xlsx"

                # Caminho completo do arquivo
                caminho_completo = os.path.join(pastas[nome_df], nome_arquivo)

                # Salvar o DataFrame em um arquivo CSV
                df.to_excel(caminho_completo, index=False)
                #print(f"DataFrame {nome_df} salvo com sucesso em: {caminho_completo}")
            except Exception as e:
                print(f"Erro ao salvar o DataFrame {nome_df}: {e}")
        print(f"Arquivo vendas de {numero_inicial_nota} ate {numero_maximo_nota} salvos com sucesso.")
        numero_inicial_nota = numero_maximo_nota + 1
    except Exception as e:
       print(f"Erro ao gerar os arquivos: {e}") 

Conexão com o banco de dados estabelecida com sucesso
Query executada com sucesso
A conexão foi bem-sucedida
Arquivo vendas de 1000 ate 1064 salvos com sucesso.
Arquivo vendas de 1065 ate 1127 salvos com sucesso.


In [ ]:
#(Otimizado) Script que gera NF, ITEMNF e VENDAS em formaato .xlsx direto na pasta para a importacao

import random
import pandas as pd
import os
from datetime import datetime
from sqlalchemy import create_engine, text
from faker import Faker

# Configurações iniciais
NUMERO_INICIAL_NOTA = 1000
VARIAVEL_CONTROLE_MAXIMO_LOOP = 1100
QUANT_MAXIMA_ITENS = 10

fake = Faker()

# Configurações do banco de dados
SERVER = "DESKTOP-HV85ADA\\PROJETO_MERCADO"
BANCO_DADOS = "MERCADO_OLTP"
DRIVER = "ODBC Driver 17 for SQL Server"
CONN_STRING = f"mssql+pyodbc://{SERVER}/{BANCO_DADOS}?driver={DRIVER}&Trusted_Connection=yes"
engine = create_engine(CONN_STRING, pool_size=10, max_overflow=30)

# Função para testar a conexão
def testar_conexao():
    try:
        with engine.connect() as conexao:
            conexao.execute(text("SELECT 1"))
            print("Conexão com o banco de dados estabelecida com sucesso.")
            return True
    except Exception as e:
        print(f"Erro ao conectar ao banco de dados: {e}")
        return False

# Função para gerar uma nota fiscal
def gerar_nota_fiscal(numero_nota, lista_produtos, data_nota):
    quant_itens = random.randint(1, QUANT_MAXIMA_ITENS)
    id_forma_pgmt = random.randint(1, 8)
    id_cliente = random.randint(1, obter_total("CLIENTE", "IDCLIENTE"))
    id_loja = random.randint(1, obter_total("LOJA", "IDLOJA"))
    id_cfop = random.randint(1, 6)
    chave_acesso = ''.join([str(random.randint(0, 9)) for _ in range(44)])
    data_criacao = data_nota.date()
    versao = 1

    itens = []
    for _ in range(quant_itens):
        produto = random.choice(lista_produtos)
        preco_unit = df_prod.loc[df_prod["NOME"] == produto, "PRECO_UNIT"].values[0]
        id_produto = df_prod.loc[df_prod["NOME"] == produto, "IDPRODUTO"].values[0]
        quantidade = random.randint(1, 10)
        total_item = round(quantidade * preco_unit, 2)
        itens.append((data_nota, numero_nota, produto, quantidade, preco_unit, total_item, 
                      id_forma_pgmt, id_cliente, id_loja, id_cfop, chave_acesso, data_criacao, 
                      "", versao, id_produto))

    total_nota = round(sum(item[5] for item in itens), 2)
    itens = [item + (total_nota,) for item in itens]

    df_nota_fiscal = pd.DataFrame(
        itens, columns=["DATA_NOTAFISCAL", "Numero_Nota", "Produto", "QUANTIDADE", "PRECO_UNIT", 
                        "TOTAL_ITEM", "ID_FORMA_PGMT", "ID_CLIENTE", "ID_LOJA", "ID_CFOP", 
                        "CHAVE_ACESSO", "DATA_CRIACAO", "DATA_ATLZ", "VERSAO", "IDPRODUTO", "TOTAL_NOTA"]
    )

    df_nota_fiscal_2 = pd.DataFrame(
        [(data_nota, id_forma_pgmt, id_cliente, id_loja, id_cfop, chave_acesso, data_criacao, 
          "", versao, total_nota)],
        columns=["DATA_NOTAFISCAL", "ID_FORMA_PGMT", "ID_CLIENTE", "ID_LOJA", "ID_CFOP", 
                 "CHAVE_ACESSO", "DATA_CRIACAO", "DATA_ATLZ", "VERSAO", "TOTAL_NOTA"]
    )

    return df_nota_fiscal, df_nota_fiscal_2

# Função para obter o total de registros em uma tabela
def obter_total(tabela, coluna):
    with engine.connect() as conexao:
        return conexao.execute(text(f"SELECT COUNT({coluna}) FROM {tabela}")).scalar()

# Função para salvar DataFrames em arquivos Excel
def salvar_dataframes(dataframes, pastas, numero_inicial_nota, numero_maximo_nota):
    id_valor = f"vendas_{numero_inicial_nota}ate_{numero_maximo_nota}"
    for nome_df, df in dataframes.items():
        try:
            nome_arquivo = f"arquivo_{nome_df}_{id_valor}.xlsx"
            caminho_completo = os.path.join(pastas[nome_df], nome_arquivo)
            df.to_excel(caminho_completo, index=False)
        except Exception as e:
            print(f"Erro ao salvar o DataFrame {nome_df}: {e}")

# Caminhos das pastas
PASTAS = {
    'ITEMNOTA': r"C:\\Users\\marce\\OneDrive\Arquivos Antigos\\Área de Trabalho\\Projeto Supermecado - Arquivos\\VENDAS\\ITEMNOTA",
    'NOTAFISCAL': r"C:\\Users\\marce\\OneDrive\Arquivos Antigos\\Área de Trabalho\\Projeto Supermecado - Arquivos\\VENDAS\\NOTAFISCAL",
    'VENDA': r"C:\\Users\\marce\\OneDrive\Arquivos Antigos\\Área de Trabalho\\Projeto Supermecado - Arquivos\\VENDAS\\VENDA"
}

# Execução principal
if testar_conexao():
    numero_inicial_nota = NUMERO_INICIAL_NOTA
    while numero_inicial_nota < VARIAVEL_CONTROLE_MAXIMO_LOOP:
        try:
            incremento = random.randint(33, 87)
            numero_maximo_nota = numero_inicial_nota + incremento
            df_prod = pd.read_sql(text("SELECT IDPRODUTO, NOME, PRECO_UNIT FROM PRODUTO"), engine)
            lista_produtos = df_prod['NOME'].to_list()

            df_notas_fiscais = []
            df_notas_fiscais_2 = []
            for numero_nota in range(numero_inicial_nota, numero_maximo_nota + 1):
                data_nota_ini = fake.date_time_between(start_date=datetime(2024, 1, 1), end_date=datetime(2024, 1, 20))
                hora_aleatoria = random.randint(7, 21)  #Gera uma hora entre 8 e 22
                minuto_aleatorio = random.randint(0, 59)  #Gera minutos aleatórios
                segundo_aleatorio = random.randint(0, 59)  #Gera segundos aleatórios
                data_nota = data_nota_ini.replace(hour=hora_aleatoria, minute=minuto_aleatorio, second=segundo_aleatorio) #Substituir o horário da data_nota pelo horário gerado
                df_nota_fiscal, df_2 = gerar_nota_fiscal(numero_nota, lista_produtos, data_nota)
                df_notas_fiscais.append(df_nota_fiscal)
                df_notas_fiscais_2.append(df_2)

            df_todas_notas = pd.concat(df_notas_fiscais, ignore_index=True)
            df_todas_notas2 = pd.concat(df_notas_fiscais_2, ignore_index=True)

            df_ITEMNOTA = df_todas_notas[['Numero_Nota', 'IDPRODUTO', 'QUANTIDADE', 'PRECO_UNIT', 'TOTAL_ITEM', 'DATA_CRIACAO', 'DATA_ATLZ', 'VERSAO']]
            df_ITEMNOTA.columns = ['ID_NOTAFISCAL', 'ID_PRODUTO', 'QUANTIDADE', 'PRECO_UNIT', 'TOTAL_ITEM', 'DATA_CRIACAO', 'DATA_ATLZ', 'VERSAO']

            df_NOTAFISCAL = df_todas_notas2[['DATA_NOTAFISCAL', 'TOTAL_NOTA', 'ID_FORMA_PGMT', 'ID_CLIENTE', 'ID_LOJA', 'ID_CFOP', 'CHAVE_ACESSO', 'DATA_CRIACAO', 'DATA_ATLZ', 'VERSAO']]
            df_NOTAFISCAL.columns = ['DATA', 'TOTAL_NOTA', 'ID_FORMA_PGMT', 'ID_CLIENTE', 'ID_LOJA', 'ID_CFOP', 'CHAVE_ACESSO', 'DATA_CRIACAO', 'DATA_ATLZ', 'VERSAO']

            df_VENDA = df_todas_notas[['DATA_NOTAFISCAL', 'Numero_Nota', 'ID_CLIENTE', 'ID_LOJA', 'ID_FORMA_PGMT', 'TOTAL_NOTA', 'DATA_CRIACAO', 'DATA_ATLZ', 'VERSAO']]
            df_VENDA.columns = ['DATA', 'ID_NOTAFISCAL', 'ID_CLIENTE', 'ID_LOJA', 'ID_FORMA_PGMT', 'TOTAL_VENDA', 'DATA_CRIACAO', 'DATA_ATLZ', 'VERSAO']

            dataframes = {
                'ITEMNOTA': df_ITEMNOTA,
                'NOTAFISCAL': df_NOTAFISCAL,
                'VENDA': df_VENDA
            }

            salvar_dataframes(dataframes, PASTAS, numero_inicial_nota, numero_maximo_nota)
            print(f"Arquivos de vendas de {numero_inicial_nota} até {numero_maximo_nota} salvos com sucesso.")
            numero_inicial_nota = numero_maximo_nota + 1
        except Exception as e:
            print(f"Erro ao gerar os arquivos: {e}")

Conexão com o banco de dados estabelecida com sucesso.


datetime.datetime(2024, 1, 11, 21, 5, 48)

2024-01-11 12:37:00


datetime.datetime(2024, 1, 11, 4, 4, 31)

2024-01-11 20:57:08


...

2024-01-12 11:37:28


datetime.datetime(2024, 1, 18, 8, 6, 52)

2024-01-18 10:35:37


datetime.datetime(2024, 1, 8, 9, 20, 46)

2024-01-08 13:01:17
Arquivos de vendas de 1092 até 1136 salvos com sucesso.


In [ ]:
#Script para gerar Tabela de Notas Fiscais, Itens de nota e Vendas(OLTP), buscando o IDPRDOUTO na tabela Produto do banco de dados

import random
import pandas as pd
import os
from datetime import datetime, date
from sqlalchemy import create_engine, VARCHAR,NUMERIC,INTEGER,DATE,DATETIME,String,text,CHAR
import numpy as np
from faker import Faker
from sqlalchemy.exc import SQLAlchemyError

# Quantidade de notas fiscais
numero_inicial_nota = 1104
numero_maximo_nota = 1145
quant_maxima_itens = 10

fake = Faker()

server = "DESKTOP-HV85ADA\\PROJETO_MERCADO"
bancodados = "MERCADO_OLTP"
driver = "ODBC Driver 17 for SQL Server"

conexaostring = f"mssql+pyodbc://{server}/{bancodados}?driver={driver}&Trusted_Connection=yes"
engine = create_engine(conexaostring)

#Função para testar a conexão
def testar_conexao_sqlalchemy(engine):
    try:        
        with engine.connect() as conexao: #Tenta conectar ao banco de dados
            print("Conexão com o banco de dados estabelecida com sucesso")
            result = conexao.execute(text("SELECT 1 AS teste")) #Executa uma consulta para testar a conexão
            print("Query executada com sucesso")
            return True
    except Exception as e:
        print(f"Erro ao conectar ao banco de dados: {e}")
        return False
#testa a conexão
if testar_conexao_sqlalchemy(engine): 
    print("A conexão foi bem-sucedida")
else:
    print("Falha na conexão.")


# Função para gerar a nota fiscal
def gerar_nota_fiscal(numero_nota, lista_produtos, quant_maxima_itens, data_nota):
    # Garantir que a quantidade de itens únicos seja pelo menos 1
    quant_item_unicos = random.randint(1, quant_maxima_itens)  # Gerar quantidade aleatória de itens únicos na nota fiscal
    #print(f"Quantidade de itens únicos na nota {numero_nota}: {quant_item_unicos}")

    lista_saida = []  # Lista para armazenar os itens da nota fiscal
    lista_NOTAFISCAL = []   

    id_forma_pgmt = random.randint(1, 8)
    total_clientes_db = (engine.connect().execute(text("SELECT COUNT(IDCLIENTE) FROM [MERCADO_OLTP].[dbo].CLIENTE"))).scalar()
    id_cliente = random.randint(1, total_clientes_db)
    total_lojas_db = (engine.connect().execute(text("SELECT COUNT(IDLOJA) FROM [MERCADO_OLTP].[dbo].LOJA"))).scalar()
    id_loja = random.randint(1, total_lojas_db)
    id_cfop = random.randint(1, 6)
    chaveacesso = ''.join([str(random.randint(0, 9)) for _ in range(44)])
    data_criacao = data_nota.date()
    data_atlz = ""
    versao = 1
    
    # Gerar os itens da nota fiscal
    for _ in range(quant_item_unicos):
        produto = random.choice(lista_produtos)
        if produto in df_prod["NOME"].values:
            preco_unit = df_prod.loc[df_prod["NOME"] == produto, "PRECO_UNIT"].values[0]
            id_produto = df_prod.loc[df_prod["NOME"] == produto, "IDPRODUTO"].values[0]
            preco_unit = round(float(preco_unit), 2)  # Preço unitário com 2 casas decimais
            quantidade = random.randint(1, 10)  # Quantidade aleatória do produto
            preco_total_item = round(quantidade * preco_unit, 2)  # Total do item
            lista_saida.append((data_nota, numero_nota, produto, quantidade, preco_unit, preco_total_item, 
                                id_forma_pgmt, id_cliente, id_loja, id_cfop, chaveacesso, data_criacao, 
                                data_atlz, versao,id_produto))            

    # Calcular o total da nota fiscal
    preco_total_nota = round(sum(item[5] for item in lista_saida), 2)
    
    # Adicionar o total da nota a cada item da lista
    lista_saida = [item + (preco_total_nota,) for item in lista_saida]
   
    lista_NOTAFISCAL.append((data_nota, id_forma_pgmt, id_cliente, id_loja, id_cfop, chaveacesso, data_criacao, 
                             data_atlz, versao,preco_total_nota))
    

    # Criar um DataFrame com os dados da nota fiscal
    df_nota_fiscal = pd.DataFrame(
        lista_saida, columns=["DATA_NOTAFISCAL", "Numero_Nota", "Produto", "QUANTIDADE", "PRECO_UNIT", "TOTAL_ITEM", 
                              "ID_FORMA_PGMT", "ID_CLIENTE", "ID_LOJA", "ID_CFOP","CHAVE_ACESSO","DATA_CRIACAO",
                              "DATA_ATLZ","VERSAO", "IDPRODUTO","TOTAL_NOTA"]
    )

    df_2 = pd.DataFrame(
        lista_NOTAFISCAL, columns=["DATA_NOTAFISCAL", "ID_FORMA_PGMT", "ID_CLIENTE", "ID_LOJA", "ID_CFOP", 
                                   "CHAVE_ACESSO", "DATA_CRIACAO", "DATA_ATLZ", "VERSAO","TOTAL_NOTA"]
    )

    
    return df_nota_fiscal, df_2

# Lista de produtos e lista de ID PRODUTO a partir do banco de dados

df_prod = pd.read_sql(text(f"SELECT IDPRODUTO, NOME, PRECO_UNIT FROM PRODUTO "),engine)

#lista_IDPRODUTO = df_prod['IDPRODUTO'].to_list()
lista_prod_DB = df_prod['NOME'].to_list()

# Lista para armazenar todas as notas fiscais
df_notas_fiscais = []
df_notas_fiscais_2 = []

# Gerar notas fiscais para o intervalo definido
for numero_nota in range(numero_inicial_nota, numero_maximo_nota + 1):
    data_nota_ini = fake.date_time_between(start_date=datetime(2024, 1, 1), end_date=datetime(2024, 1, 20))
    hora_aleatoria = random.randint(7, 21)  #Gera uma hora entre 8 e 22
    minuto_aleatorio = random.randint(0, 59)  #Gera minutos aleatórios
    segundo_aleatorio = random.randint(0, 59)  #Gera segundos aleatórios
    data_nota = data_nota_ini.replace(hour=hora_aleatoria, minute=minuto_aleatorio, second=segundo_aleatorio) #Substituir o horário da data_nota pelo horário gerado
    df_nota_fiscal, df_2 = gerar_nota_fiscal(numero_nota, lista_prod_DB, quant_maxima_itens, data_nota)
    df_notas_fiscais.append(df_nota_fiscal)
    df_notas_fiscais_2.append(df_2)

# Concatenar todas as notas fiscais em um único DataFrame
df_todas_notas = pd.concat(df_notas_fiscais, ignore_index=True)
df_todas_notas2 = pd.concat(df_notas_fiscais_2, ignore_index=True)

df_ITEMNOTA = pd.DataFrame({
                'ID_NOTAFISCAL':  df_todas_notas['Numero_Nota'],
                'ID_PRODUTO':     df_todas_notas['IDPRODUTO'],
                'QUANTIDADE':     df_todas_notas['QUANTIDADE'],
                'PRECO_UNIT':     df_todas_notas['PRECO_UNIT'],
                'TOTAL_ITEM':     df_todas_notas['TOTAL_ITEM'],
                'DATA_CRIACAO':   df_todas_notas['DATA_CRIACAO'],
                'DATA_ATLZ':      df_todas_notas['DATA_ATLZ'],
                'VERSAO':         df_todas_notas['VERSAO'],
})

df_NOTAFISCAL = pd.DataFrame({
                'DATA':           df_todas_notas2['DATA_NOTAFISCAL'],
                'TOTAL_NOTA':     df_todas_notas2['TOTAL_NOTA'],
                'ID_FORMA_PGMT':  df_todas_notas2['ID_FORMA_PGMT'],
                'ID_CLIENTE':     df_todas_notas2['ID_CLIENTE'],
                'ID_LOJA':        df_todas_notas2['ID_LOJA'],
                'ID_CFOP':        df_todas_notas2['ID_CFOP'],
                'CHAVE_ACESSO':   df_todas_notas2['CHAVE_ACESSO'],
                'DATA_CRIACAO':   df_todas_notas2['DATA_CRIACAO'],
                'DATA_ATLZ':      df_todas_notas2['DATA_ATLZ'],
                'VERSAO':         df_todas_notas2['VERSAO'],
})

df_VENDA = pd.DataFrame({
                'DATA':           df_todas_notas['DATA_NOTAFISCAL'],
                'ID_NOTAFISCAL':  df_todas_notas['Numero_Nota'],
                'ID_CLIENTE':     df_todas_notas['ID_CLIENTE'],
                'ID_LOJA':        df_todas_notas['ID_LOJA'],
                'ID_FORMA_PGMT':  df_todas_notas['ID_FORMA_PGMT'],
                'TOTAL_VENDA':     df_todas_notas['TOTAL_NOTA'],
                'DATA_CRIACAO':   df_todas_notas['DATA_CRIACAO'],
                'DATA_ATLZ':      df_todas_notas['DATA_ATLZ'],
                'VERSAO':         df_todas_notas['VERSAO'],
})

caminho = r"C:\\Users\\marce\\OneDrive\Arquivos Antigos\\Área de Trabalho\\Projeto Supermecado - Arquivos\\VENDAS"
path_ITEMNOTA = r"\\ITEMNOTA\\Novo(a) Documento de Texto.csv"

# Caminhos das pastas
pastas = {
    'ITEMNOTA': r"C:\\Users\\marce\\OneDrive\Arquivos Antigos\\Área de Trabalho\\Projeto Supermecado - Arquivos\\VENDAS\\ITEMNOTA",
    'NOTAFISCAL': r"C:\\Users\\marce\\OneDrive\Arquivos Antigos\\Área de Trabalho\\Projeto Supermecado - Arquivos\\VENDAS\\NOTAFISCAL",
    'VENDA': r"C:\\Users\\marce\\OneDrive\Arquivos Antigos\\Área de Trabalho\\Projeto Supermecado - Arquivos\\VENDAS\\VENDA"
}
# Dicionário com os DataFrames e suas pastas correspondentes
dataframes = {
    'ITEMNOTA': df_ITEMNOTA,
    'NOTAFISCAL': df_NOTAFISCAL,
    'VENDA': df_VENDA
}

# Salvar cada DataFrame em um arquivo CSV na pasta correspondente
for nome_df, df in dataframes.items():
    try:
        id_valor = f"vendas_{numero_inicial_nota}ate_{numero_maximo_nota}"
        # Nome do arquivo
        nome_arquivo = f"arquivo_{nome_df}_{id_valor}.xlsx"
        
        # Caminho completo do arquivo
        caminho_completo = os.path.join(pastas[nome_df], nome_arquivo)
        
        # Salvar o DataFrame em um arquivo CSV
        df.to_excel(caminho_completo, index=False)
        print(f"DataFrame {nome_df} salvo com sucesso em: {caminho_completo}")
    except Exception as e:
        print(f"Erro ao salvar o DataFrame {nome_df}: {e}")

In [ ]:
#Script que gera NF, ITEMNF e VENDAS em formaato .xlsx direto na pasta para a importacao

import random
import pandas as pd
import os
from datetime import datetime, date
from sqlalchemy import create_engine, VARCHAR,NUMERIC,INTEGER,DATE,DATETIME,String,text,CHAR
import numpy as np
from faker import Faker

# Quantidade de notas fiscais

numero_inicial_nota = 1000
numero_maximo_nota = 3001
quant_maxima_itens = 100
variavel_controle_maximo_LOOP = 1300 #Valor maximo que da nota inicial dentro do loop de geracao de NF's

fake = Faker()

server = "DESKTOP-HV85ADA\\PROJETO_MERCADO"
bancodados = "MERCADO_OLTP"
driver = "ODBC Driver 17 for SQL Server"

conexaostring = f"mssql+pyodbc://{server}/{bancodados}?driver={driver}&Trusted_Connection=yes"
engine = create_engine(conexaostring)

#Função para testar a conexão
def testar_conexao_sqlalchemy(engine):
    try:        
        with engine.connect() as conexao: #Tenta conectar ao banco de dados
            print("Conexão com o banco de dados estabelecida com sucesso")
            result = conexao.execute(text("SELECT 1 AS teste")) #Executa uma consulta para testar a conexão
            print("Query executada com sucesso")
            return True
    except Exception as e:
        print(f"Erro ao conectar ao banco de dados: {e}")
        return False
#testa a conexão
if testar_conexao_sqlalchemy(engine): 
    print("A conexão foi bem-sucedida")
else:
    print("Falha na conexão.")

# Função para gerar a nota fiscal
def gerar_nota_fiscal(numero_nota, lista_produtos, quant_maxima_itens, data_nota):
    # Garantir que a quantidade de itens únicos seja pelo menos 1
    quant_item_unicos = random.randint(1, quant_maxima_itens)  # Gerar quantidade aleatória de itens únicos na nota fiscal
    #print(f"Quantidade de itens únicos na nota {numero_nota}: {quant_item_unicos}")

    lista_saida = []  # Lista para armazenar os itens da nota fiscal
    lista_NOTAFISCAL = []   

    id_forma_pgmt = random.randint(1, 8)
    total_clientes_db = (engine.connect().execute(text("SELECT COUNT(IDCLIENTE) FROM [MERCADO_OLTP].[dbo].CLIENTE"))).scalar()
    id_cliente = random.randint(1, total_clientes_db)
    total_lojas_db = (engine.connect().execute(text("SELECT COUNT(IDLOJA) FROM [MERCADO_OLTP].[dbo].LOJA"))).scalar()
    id_loja = random.randint(1, total_lojas_db)
    id_cfop = random.randint(1, 6)
    chaveacesso = ''.join([str(random.randint(0, 9)) for _ in range(44)])
    data_criacao = data_nota.date()
    data_atlz = ""
    versao = 1
    
    # Gerar os itens da nota fiscal
    for _ in range(quant_item_unicos):
        produto = random.choice(lista_produtos)
        if produto in df_prod["NOME"].values:
            preco_unit = df_prod.loc[df_prod["NOME"] == produto, "PRECO_UNIT"].values[0]
            id_produto = df_prod.loc[df_prod["NOME"] == produto, "IDPRODUTO"].values[0]
            preco_unit = round(float(preco_unit), 2)  # Preço unitário com 2 casas decimais
            quantidade = random.randint(1, 10)  # Quantidade aleatória do produto
            preco_total_item = round(quantidade * preco_unit, 2)  # Total do item
            lista_saida.append((data_nota, numero_nota, produto, quantidade, preco_unit, preco_total_item, 
                                id_forma_pgmt, id_cliente, id_loja, id_cfop, chaveacesso, data_criacao, 
                                data_atlz, versao,id_produto))            

    # Calcular o total da nota fiscal
    preco_total_nota = round(sum(item[5] for item in lista_saida), 2)
    
    # Adicionar o total da nota a cada item da lista
    lista_saida = [item + (preco_total_nota,) for item in lista_saida]
   
    lista_NOTAFISCAL.append((data_nota, id_forma_pgmt, id_cliente, id_loja, id_cfop, chaveacesso, data_criacao, 
                             data_atlz, versao,preco_total_nota))
    

    # Criar um DataFrame com os dados da nota fiscal
    df_nota_fiscal = pd.DataFrame(
        lista_saida, columns=["DATA_NOTAFISCAL", "Numero_Nota", "Produto", "QUANTIDADE", "PRECO_UNIT", "TOTAL_ITEM", 
                              "ID_FORMA_PGMT", "ID_CLIENTE", "ID_LOJA", "ID_CFOP","CHAVE_ACESSO","DATA_CRIACAO",
                              "DATA_ATLZ","VERSAO", "IDPRODUTO","TOTAL_NOTA"]
    )

    df_2 = pd.DataFrame(
        lista_NOTAFISCAL, columns=["DATA_NOTAFISCAL", "ID_FORMA_PGMT", "ID_CLIENTE", "ID_LOJA", "ID_CFOP", 
                                   "CHAVE_ACESSO", "DATA_CRIACAO", "DATA_ATLZ", "VERSAO","TOTAL_NOTA"]
    )

    
    return df_nota_fiscal, df_2


while numero_inicial_nota < variavel_controle_maximo_LOOP:
    try:
        
        incremento = random.randint(33, 87)
        numero_maximo_nota = numero_inicial_nota + incremento
        df_prod = pd.read_sql(text(f"SELECT IDPRODUTO, NOME, PRECO_UNIT FROM PRODUTO "),engine)
        lista_prod_DB = df_prod['NOME'].to_list()
        df_notas_fiscais = []
        df_notas_fiscais_2 = []
        for numero_nota in range(numero_inicial_nota, numero_maximo_nota + 1):
            data_nota_ini = fake.date_time_between(start_date=datetime(2024, 1, 1), end_date=datetime(2024, 1, 20))
            hora_aleatoria = random.randint(7, 21)  #Gera uma hora entre 8 e 22
            minuto_aleatorio = random.randint(0, 59)  #Gera minutos aleatórios
            segundo_aleatorio = random.randint(0, 59)  #Gera segundos aleatórios
            data_nota = data_nota_ini.replace(hour=hora_aleatoria, minute=minuto_aleatorio, second=segundo_aleatorio) #Substituir o horário da data_nota pelo horário gerado
            df_nota_fiscal, df_2 = gerar_nota_fiscal(numero_nota, lista_prod_DB, quant_maxima_itens, data_nota)
            df_notas_fiscais.append(df_nota_fiscal)
            df_notas_fiscais_2.append(df_2)
        df_todas_notas = pd.concat(df_notas_fiscais, ignore_index=True)
        df_todas_notas2 = pd.concat(df_notas_fiscais_2, ignore_index=True)
        df_ITEMNOTA = pd.DataFrame({
                    'ID_NOTAFISCAL':  df_todas_notas['Numero_Nota'],
                    'ID_PRODUTO':     df_todas_notas['IDPRODUTO'],
                    'QUANTIDADE':     df_todas_notas['QUANTIDADE'],
                    'PRECO_UNIT':     df_todas_notas['PRECO_UNIT'],
                    'TOTAL_ITEM':     df_todas_notas['TOTAL_ITEM'],
                    'DATA_CRIACAO':   df_todas_notas['DATA_CRIACAO'],
                    'DATA_ATLZ':      df_todas_notas['DATA_ATLZ'],
                    'VERSAO':         df_todas_notas['VERSAO'],
        })
        df_NOTAFISCAL = pd.DataFrame({
                        'DATA':           df_todas_notas2['DATA_NOTAFISCAL'],
                        'TOTAL_NOTA':     df_todas_notas2['TOTAL_NOTA'],
                        'ID_FORMA_PGMT':  df_todas_notas2['ID_FORMA_PGMT'],
                        'ID_CLIENTE':     df_todas_notas2['ID_CLIENTE'],
                        'ID_LOJA':        df_todas_notas2['ID_LOJA'],
                        'ID_CFOP':        df_todas_notas2['ID_CFOP'],
                        'CHAVE_ACESSO':   df_todas_notas2['CHAVE_ACESSO'],
                        'DATA_CRIACAO':   df_todas_notas2['DATA_CRIACAO'],
                        'DATA_ATLZ':      df_todas_notas2['DATA_ATLZ'],
                        'VERSAO':         df_todas_notas2['VERSAO'],
        })
        df_VENDA = pd.DataFrame({
                        'DATA':           df_todas_notas['DATA_NOTAFISCAL'],
                        'ID_NOTAFISCAL':  df_todas_notas['Numero_Nota'],
                        'ID_CLIENTE':     df_todas_notas['ID_CLIENTE'],
                        'ID_LOJA':        df_todas_notas['ID_LOJA'],
                        'ID_FORMA_PGMT':  df_todas_notas['ID_FORMA_PGMT'],
                        'TOTAL_VENDA':     df_todas_notas['TOTAL_NOTA'],
                        'DATA_CRIACAO':   df_todas_notas['DATA_CRIACAO'],
                        'DATA_ATLZ':      df_todas_notas['DATA_ATLZ'],
                        'VERSAO':         df_todas_notas['VERSAO'],
        })
        caminho = r"C:\\Users\\marce\\OneDrive\Arquivos Antigos\\Área de Trabalho\\Projeto Supermecado - Arquivos\\VENDAS"
        path_ITEMNOTA = r"\\ITEMNOTA\\Novo(a) Documento de Texto.csv"

        # Caminhos das pastas
        pastas = {
            'ITEMNOTA': r"C:\\Users\\marce\\OneDrive\Arquivos Antigos\\Área de Trabalho\\Projeto Supermecado - Arquivos\\VENDAS\\ITEMNOTA",
            'NOTAFISCAL': r"C:\\Users\\marce\\OneDrive\Arquivos Antigos\\Área de Trabalho\\Projeto Supermecado - Arquivos\\VENDAS\\NOTAFISCAL",
            'VENDA': r"C:\\Users\\marce\\OneDrive\Arquivos Antigos\\Área de Trabalho\\Projeto Supermecado - Arquivos\\VENDAS\\VENDA"
        }
        # Dicionário com os DataFrames e suas pastas correspondentes
        dataframes = {
            'ITEMNOTA': df_ITEMNOTA,
            'NOTAFISCAL': df_NOTAFISCAL,
            'VENDA': df_VENDA
        }
        # Salvar cada DataFrame em um arquivo CSV na pasta correspondente
        for nome_df, df in dataframes.items():
            try:
                id_valor = f"vendas_{numero_inicial_nota}ate_{numero_maximo_nota}"
                # Nome do arquivo
                nome_arquivo = f"arquivo_{nome_df}_{id_valor}.xlsx"

                # Caminho completo do arquivo
                caminho_completo = os.path.join(pastas[nome_df], nome_arquivo)

                # Salvar o DataFrame em um arquivo CSV
                df.to_excel(caminho_completo, index=False)
                #print(f"DataFrame {nome_df} salvo com sucesso em: {caminho_completo}")
            except Exception as e:
                print(f"Erro ao salvar o DataFrame {nome_df}: {e}")
        print(f"Arquivo vendas de {numero_inicial_nota} ate {numero_maximo_nota} salvos com sucesso.")
        numero_inicial_nota = numero_maximo_nota + 1
    except Exception as e:
       print(f"Erro ao gerar os arquivos: {e}") 

Conexão com o banco de dados estabelecida com sucesso
Query executada com sucesso
A conexão foi bem-sucedida
Arquivo vendas de 1000 ate 1039 salvos com sucesso.
Arquivo vendas de 1040 ate 1087 salvos com sucesso.
Arquivo vendas de 1088 ate 1144 salvos com sucesso.
Arquivo vendas de 1145 ate 1195 salvos com sucesso.
Arquivo vendas de 1196 ate 1274 salvos com sucesso.
Arquivo vendas de 1275 ate 1357 salvos com sucesso.


In [ ]:
#Script para enviar arquivos da pasta Vendas par o Sql server

import random
import pandas as pd
from datetime import datetime, date
from sqlalchemy import create_engine, VARCHAR,NUMERIC,INTEGER,DATE,DATETIME,String,text,CHAR
import numpy as np
from faker import Faker
from sqlalchemy.exc import SQLAlchemyError
import os

fake = Faker()

server = "DESKTOP-HV85ADA\\PROJETO_MERCADO"
bancodados = "MERCADO_OLTP"
driver = "ODBC Driver 17 for SQL Server"

conexaostring = f"mssql+pyodbc://{server}/{bancodados}?driver={driver}&Trusted_Connection=yes"
engine = create_engine(conexaostring)

#Função para testar a conexão
def testar_conexao_sqlalchemy(engine):
    try:        
        with engine.connect() as conexao: #Tenta conectar ao banco de dados
            print("Conexão com o banco de dados estabelecida com sucesso")
            result = conexao.execute(text("SELECT 1 AS teste")) #Executa uma consulta para testar a conexão
            print("Query executada com sucesso")
            return True
    except Exception as e:
        print(f"Erro ao conectar ao banco de dados: {e}")
        return False
#testa a conexão
if testar_conexao_sqlalchemy(engine): 
    print("A conexão foi bem-sucedida")
else:
    print("Falha na conexão.")

def ler_arquivos_xlsx(caminho_pasta):
    dataframes = []
    for arquivo in os.listdir(caminho_pasta):
        if arquivo.endswith(".xlsx"):
            try:
                caminho_completo = os.path.join(caminho_pasta, arquivo)
                df = pd.read_excel(caminho_completo)
                dataframes.append(df)
                print(f"Arquivo {arquivo} lido com sucesso.")
            except Exception as e:
                print(f"Erro ao ler o arquivo {arquivo}: {e}")
    return pd.concat(dataframes, ignore_index=True) if dataframes else pd.DataFrame()

#Caminhos das pastas
caminho_itemnota = r"C:\Users\marce\OneDrive\Arquivos Antigos\Área de Trabalho\Projeto Supermecado - Arquivos\VENDAS\ITEMNOTA"
caminho_notafiscal = r"C:\Users\marce\OneDrive\Arquivos Antigos\Área de Trabalho\Projeto Supermecado - Arquivos\VENDAS\NOTAFISCAL"
caminho_venda = r"C:\Users\marce\OneDrive\Arquivos Antigos\Área de Trabalho\Projeto Supermecado - Arquivos\VENDAS\VENDA"

#Lê os arquivos e cria os DataFrames
df_ITEMNOTA = ler_arquivos_xlsx(caminho_itemnota)
df_NOTAFISCAL = ler_arquivos_xlsx(caminho_notafiscal)
df_VENDA = ler_arquivos_xlsx(caminho_venda)

#Exibe os DataFrames
print("\nDataFrame ITEMNOTA:")
display(df_ITEMNOTA)

print("\nDataFrame NOTAFISCAL:")
display(df_NOTAFISCAL)

print("\nDataFrame VENDA:")
display(df_VENDA)
